In [107]:
import pandas as pd
import folium

In [108]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

In [109]:
Selected_zones = pd.read_csv('C:\Users\pelihov\Week_4_Selected_zones.csv')
Selected_zones = Selected_zones.drop(['Unnamed: 0','geometry','mean_tr_qty'], axis = 1)
Selected_zones.head()
Selected_zones['centr_lon'] = (Selected_zones['west']+Selected_zones['east'])/2
Selected_zones['centr_lat'] = (Selected_zones['south']+Selected_zones['north'])/2
Selected_zones['zone_id'] = Selected_zones['Zone_id']
Selected_zones = Selected_zones.drop(['Zone_id','Trips_qty'], axis =1)
Selected_zones.head()

,west,east,south,north,centr_lon,centr_lat,zone_id
0,-74.022246,-74.011135,40.697437,40.705825,-74.016691,40.701631,1075
1,-74.022246,-74.011135,40.705825,40.714213,-74.016691,40.710019,1076
2,-74.022246,-74.011135,40.714213,40.722601,-74.016691,40.718407,1077
3,-74.011135,-74.000023,40.697437,40.705825,-74.005579,40.701631,1125
4,-74.011135,-74.000023,40.705825,40.714213,-74.005579,40.710019,1126


In [119]:
trips_data = pd.read_csv('C:\Users\pelihov\Downloads/june_trips_forecast.csv')
                        #parse_dates = ['start_time'])

In [120]:
trips_data['start_time'] = pd.to_datetime(trips_data['start_time'])

In [121]:
trips_data = trips_data.drop(['Unnamed: 0'], axis =1)

In [122]:
#trips_data = trips_data.drop(trips_data.columns[2:-12], axis =1)

In [123]:
trips_data.head()

,start_time,zone_id,trips_qty_1,trips_qty_2,trips_qty_3,trips_qty_4,trips_qty_5,trips_qty_6,Trips_qty_0f_norm,Trips_qty_1f_norm,Trips_qty_2f_norm,Trips_qty_3f_norm,Trips_qty_4f_norm,Trips_qty_5f_norm
0,2016-05-31 18:00:00,1076,59.0,48.0,74.0,36.0,23.0,11.0,36.549074,32.219962,30.072580,29.781128,30.605976,32.563599
1,2016-05-31 18:00:00,1783,27.0,26.0,32.0,39.0,17.0,17.0,37.727168,42.163600,46.187005,45.704646,43.380572,35.237719
2,2016-05-31 18:00:00,1332,71.0,68.0,68.0,42.0,23.0,27.0,66.989339,56.087718,45.854321,39.977451,34.392404,32.959357
3,2016-05-31 18:00:00,1783,142.0,121.0,148.0,130.0,70.0,41.0,154.792081,170.454512,194.600133,207.401281,223.601202,204.124325
4,2016-05-31 18:00:00,1384,94.0,67.0,51.0,45.0,26.0,18.0,61.223449,49.035861,38.928237,32.227711,28.254958,29.679169


In [118]:
#file_name = 'C:\Users\pelihov\Downloads/june_trips_forecast.csv'

#trips_data.to_csv(file_name, sep=',')

In [101]:
def get_trip_counts_by_date_hour(selected_date_hour = '2016-05-31 18:00:00'):
        #select one time of day
    subset = trips_data[trips_data["start_time"]==selected_date_hour]
    

    #join departure counts, arrival counts, and locations
    trip_counts = subset.merge(Selected_zones, how = 'inner', on = ['zone_id'])
    return trip_counts

# print a sample to check our code works
#get_trip_counts_by_date_hour().head()

In [110]:
def plot_station_counts(trip_counts):
    # generate a new map
    folium_map = folium.Map(location=[40.738, -73.98],
                            zoom_start=13,
                            tiles="CartoDB dark_matter",
                            width='50%')

    # for each row in the data, add a cicle marker
    for index, row in trip_counts.iterrows():
        # calculate net departures
        net_departures = row["Trips_qty_1f_norm"]
        
        # generate the popup message that is shown on click.
        popup_text = "{}<br> zone_id total departures: {}"
        popup_text = popup_text.format(row["zone_id"],
                              net_departures)
        
        # radius of circles
        radius = net_departures/20
        
        # choose the color of the marker
        if net_departures>0:
            color="#E37222" # tangerine
        else:
            color="#0A8A9F" # teal
        
        # add marker to the map
        folium.CircleMarker(location=(row["centr_lat"],
                                      row["centr_lon"]),
                            radius=radius,
                            color=color,
                            popup=popup_text,
                            fill=True).add_to(folium_map)
    return folium_map

In [104]:
# plot net departures at 9AM
trip_counts = get_trip_counts_by_date_hour()
#trip_counts.head()

In [105]:
plot_station_counts(trip_counts)

In [51]:
folium_map.save("part_1.html")

In [37]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
from IPython.html import widgets

In [95]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
    
def show_plot(zone_id = 1378, forecast_shift = 1):
    algo = "Lasso"
    column_name_y = 'trips_qty_'+str(forecast_shift)
    y_train = trips_data[trips_data.zone_id == zone_id][column_name_y]
        
    column_name_pred =  'Trips_qty_' + str(forecast_shift-1) + 'f_norm'
    prediction = trips_data[trips_data.zone_id == zone_id][column_name_pred]
    
    MAE = mean_absolute_error(prediction, y_train)

    plt.figure(figsize=(20, 4))
    plt.plot(prediction, "r", label="prediction", alpha = 0.4)
    plt.legend(loc="best")
    plt.title("{} prediction\n MAE {} count trip".format(algo, MAE))
    plt.show()
    
    plt.figure(figsize=(20, 4))
    plt.plot(y_train.values, label="actual", alpha = 0.4)
    plt.legend(loc="best")
    plt.title("Actual values count trip")
    plt.show()

    plt.show()

In [96]:
interact(show_plot, zone_id=Selected_zones.zone_id, forecast_shift = range(1,6,1));

A Jupyter Widget